In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import matplotlib.pylab as plt
import PyPDF2
import string
import re
import os
import pickle
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from nltk import word_tokenize
pd.set_option('display.max_columns', 100)

In [2]:
with open('dictionary.pickle', 'rb') as handle:
    b = pickle.load(handle)
Cases = pd.read_csv('data/Cases2018.csv', encoding='iso-8859-1')

In [3]:
#key_to_value_lengths = {k:len(v) for k, v in b.items()}
newdict = {k: v for k, v in b.items() if len(v) > 60000}

In [4]:
Labels = Cases[['docket','issueArea']]
Text = pd.DataFrame.from_dict(newdict, orient="index")

In [5]:
Text = Text.rename(index=str, columns={0: "Document"})

In [6]:
Text['docket'] = Text.index
Text.reset_index(drop=True)

,Document,docket
0,1 IN THE SU...,00-1011
1,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1021
2,1 2 \n ...,00-1045
3,12345678910\n111213141516171819202122232425 IN...,00-10666
4,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1073
5,1234\n5\n678\n910\n11\n12\n131415\n161718\n19\...,00-1089
6,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1167
7,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1187
8,1 IN THE SU...,00-121
9,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1214


In [7]:
Text = pd.merge(Text, Labels, on='docket', how='inner')
Text.dropna(inplace=True)
#Text.drop(Text[Text.issueArea>=13.0].index, inplace = True)

In [8]:
Labels = Text[['issueArea']]
Text.drop('issueArea', axis=1, inplace= True)

In [9]:
user_defined_stop_words = ['-x','--','x•','n','w','e','washington', 'monday','tuesday','x','v','alderson','petitioner','respondent','reporting','company','supreme','court','united','states','states•','•v','•petitioner'] 

i = stopwords.words('english')
j = list(string.punctuation) + user_defined_stop_words

stoplist = set(i).union(j)

In [10]:
Text['Document'] = Text['Document'].apply(lambda x: " ".join(x.lower() for x in x.split()))
Text['Document'] = Text['Document'].str.replace('[^\w\s]',' ')
Text['Document'] = Text['Document'].apply(lambda x: " ".join(x for x in x.split() if x not in stoplist and not x[0].isdigit()))


In [11]:
from textblob import Word
Text['Document'] = Text['Document'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
Text['Document'].head()

0    deboris calcano martinez et al petitioner immi...
1    rush prudential hmo inc deborah c moran c wedn...
2    trw inc adelaide andrew c october entitled mat...
3    william joseph harris c march entitled matter ...
4    owasso independent school district aka owasso ...
Name: Document, dtype: object

In [12]:
X_train, X_test, y_train, y_test = train_test_split(Text.Document, Labels, test_size=0.25, random_state=23)
y_test = np.ravel(y_test)
y_train = np.ravel(y_train)

In [24]:
vectorizer = TfidfVectorizer(ngram_range=(2, 4), min_df = 2, max_df = .35)
X_train_tfidf = vectorizer.fit_transform(X_train)

In [ ]:
#sm = SMOTE(ratio=1,random_state=12)
#X_resampled, y_resampled = SMOTE().fit_resample(X_train_tfidf, y_train)

In [25]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=.001).fit(X_train_tfidf, y_train)

In [26]:
X_new_tfidf = vectorizer.transform(X_test)

predicted = clf.predict(X_new_tfidf)

In [27]:
score2 = metrics.accuracy_score(y_test, predicted)
print(score2)

0.6408163265306123


In [ ]:
clf = SGDClassifier(loss='log', penalty='l2',alpha=.001, random_state=23, n_jobs=-1, n_iter = 8200)

In [ ]:
mod2 =  clf.fit(X_resampled, y_resampled)
predicted_svm = mod2.predict(X_new_tfidf)
score2 = metrics.accuracy_score(y_test, predicted_svm)
print("accuracy:   %0.3f" % score2)

In [18]:
Labels['issueArea'].value_counts()

1.0     259
8.0     192
2.0     161
9.0     129
10.0     59
3.0      58
4.0      30
5.0      21
7.0      19
12.0     18
6.0      17
11.0      9
13.0      4
14.0      1
Name: issueArea, dtype: int64

In [ ]:
y_train

In [19]:
y_test

array([ 8.,  2.,  1.,  1.,  4.,  8.,  8.,  1.,  1.,  9.,  8.,  8.,  3.,
        1.,  9.,  9., 10.,  8.,  8., 12.,  3.,  4.,  8.,  2.,  3.,  2.,
        1.,  4.,  6.,  8.,  2.,  7.,  1.,  8.,  1.,  8.,  2.,  3.,  8.,
        8.,  1.,  8., 10.,  8.,  2.,  8.,  4.,  9.,  4.,  1.,  8.,  1.,
        2.,  5.,  8., 10.,  1.,  8.,  2., 10.,  9.,  1.,  2.,  4.,  2.,
        9.,  1., 10.,  1.,  5.,  3.,  3., 10.,  9.,  8., 12.,  2.,  1.,
        2., 10.,  8.,  4.,  3.,  2.,  2.,  9.,  9.,  9., 10.,  9.,  1.,
        5.,  9.,  1.,  2.,  1.,  2.,  4.,  1.,  2.,  1.,  8.,  1.,  9.,
       10.,  8., 12.,  8.,  9.,  1.,  2.,  1.,  1., 10.,  9.,  9.,  4.,
        2.,  5.,  8.,  2.,  6.,  8.,  2., 10.,  2.,  6.,  1.,  1.,  1.,
        8.,  8.,  5.,  5.,  1.,  1.,  8.,  1.,  2., 11.,  3.,  9.,  8.,
        8.,  2.,  9.,  9.,  8.,  1.,  1.,  2.,  8.,  7.,  1.,  2.,  1.,
       11.,  5.,  8.,  3.,  9.,  8.,  2.,  6.,  9.,  8.,  2.,  9.,  8.,
        5.,  2.,  9.,  8.,  1.,  1., 13.,  2.,  8.,  3.,  1.,  1

In [17]:
predicted

array([ 8.,  1.,  1.,  1.,  8.,  8.,  8.,  1.,  1.,  8.,  8.,  8.,  3.,
        1.,  8.,  8., 10.,  9.,  8.,  8.,  3.,  1.,  9.,  2.,  3.,  2.,
        1.,  1.,  8.,  8.,  2.,  8.,  1.,  8.,  1.,  2.,  2.,  3.,  8.,
        8.,  1.,  8.,  8.,  8.,  2.,  6.,  1.,  8.,  8.,  1.,  8.,  1.,
        1.,  2.,  8.,  8.,  8.,  8.,  1., 10.,  8.,  1.,  2.,  9.,  2.,
        1.,  1.,  8.,  1.,  6.,  2.,  3.,  8.,  9.,  8.,  8.,  2.,  1.,
        2., 10.,  8.,  8.,  3.,  1.,  2.,  1.,  1.,  2., 10.,  8.,  1.,
        1.,  9.,  1.,  9.,  1.,  2.,  4.,  1.,  2.,  1.,  8.,  1.,  2.,
        1., 12., 12.,  8.,  3.,  1.,  9.,  1.,  1., 10.,  1.,  9.,  4.,
        9.,  8.,  8.,  2.,  8.,  8.,  2.,  8.,  2.,  6.,  1.,  1.,  1.,
        8., 10., 10.,  6.,  1.,  1.,  8.,  1.,  9., 11.,  3.,  9.,  9.,
        8.,  2.,  1.,  9.,  8.,  1.,  1.,  1.,  8.,  7.,  1.,  2.,  1.,
       11.,  8.,  8.,  2.,  2.,  1.,  2.,  8.,  8.,  8.,  2.,  9.,  8.,
        5.,  2.,  8.,  9.,  1.,  1.,  8.,  2.,  8.,  3.,  1.,  1